In [137]:
import pandas as pd
import os
%xmode minimal
pd.set_option("mode.copy_on_write", True)
import seaborn as sns

Exception reporting mode: Minimal


In [138]:
data = {
    "country": ['Afghanistan', 'Afghanistan', 'Brazil', 'Brazil', 'China', 'China'],
    "year": [1999, 2000, 1999, 2000, 1999, 2000],
    "cases": [745, 2666, 37737, 80488, 212258, 213766],
    "population": [ 19987071, 20595360, 172006362, 174504898, 1272915272, 1280428583]
    }
df = pd.DataFrame(data).convert_dtypes()
df

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


In [139]:
df_melt =df.melt(id_vars=["country", "year"])
df_melt

,country,year,variable,value
0,Afghanistan,1999,cases,745
1,Afghanistan,2000,cases,2666
2,Brazil,1999,cases,37737
3,Brazil,2000,cases,80488
4,China,1999,cases,212258
5,China,2000,cases,213766
6,Afghanistan,1999,population,19987071
7,Afghanistan,2000,population,20595360
8,Brazil,1999,population,172006362
9,Brazil,2000,population,174504898


In [140]:
df_melt.pivot(index=["country", "year"], columns="variable", values="value").reset_index()

variable,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


In [141]:
# pivot() "selbst bauen"
df_melt.groupby(by=["country", "year", "variable"])["value"].sum().unstack().reset_index()

variable,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


In [142]:
df # Zurück zum df

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


In [143]:
# hier könnten wir auch pivot nach year machen
df_p =df.pivot(index=["country"], columns="year", values=["cases", "population"]).reset_index()


In [144]:
insert = pd.DataFrame(data={"country": ['Afghanistan'], "year": [1999], "cases":[10], "population": [42]}, index=[0])

In [145]:
insert

,country,year,cases,population
0,Afghanistan,1999,10,42


In [146]:
pd.concat([df_melt, insert])

,country,year,variable,value,cases,population
0,Afghanistan,1999,cases,745,NaN,NaN
1,Afghanistan,2000,cases,2666,NaN,NaN
2,Brazil,1999,cases,37737,NaN,NaN
3,Brazil,2000,cases,80488,NaN,NaN
4,China,1999,cases,212258,NaN,NaN
5,China,2000,cases,213766,NaN,NaN
6,Afghanistan,1999,population,19987071,NaN,NaN
7,Afghanistan,2000,population,20595360,NaN,NaN
8,Brazil,1999,population,172006362,NaN,NaN
9,Brazil,2000,population,174504898,NaN,NaN


In [147]:
df_melt_add = pd.concat([df_melt, insert])

In [148]:
# pivot() auf dem df_melt, wo eine nicht-eindeutige Zeile ergänzt wurdel
#Sollte eigentlich nicht funktionieren
# --> Zei einträge 1999 für afgahnistan
df_melt_add.pivot(index=["country", "year"], columns="variable", values="value").reset_index()

variable,country,year,NaN,cases,population
0,Afghanistan,1999,NaN,745,19987071
1,Afghanistan,2000,NaN,2666,20595360
2,Brazil,1999,NaN,37737,172006362
3,Brazil,2000,NaN,80488,174504898
4,China,1999,NaN,212258,1272915272
5,China,2000,NaN,213766,1280428583


In [149]:
df_melt_add.pivot_table(index=["country", "year"], columns="variable", values="value", aggfunc= 'sum') .reset_index()

variable,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


stack / unstack

In [150]:
df = df.set_index(["country", "year", "cases"])

In [151]:
df.unstack()

population                                              \
cases                745       2666       37737      80488       212258   
country     year                                                          
Afghanistan 1999   19987071      <NA>       <NA>       <NA>        <NA>   
            2000       <NA>  20595360       <NA>       <NA>        <NA>   
Brazil      1999       <NA>      <NA>  172006362       <NA>        <NA>   
            2000       <NA>      <NA>       <NA>  174504898        <NA>   
China       1999       <NA>      <NA>       <NA>       <NA>  1272915272   
            2000       <NA>      <NA>       <NA>       <NA>        <NA>   

                              
cases                 213766  
country     year              
Afghanistan 1999        <NA>  
            2000        <NA>  
Brazil      1999        <NA>  
            2000        <NA>  
China       1999        <NA>  
            2000  1280428583

In [152]:
df.unstack().unstack().stack().stack()

population
country     year cases             
Afghanistan 1999 745       19987071
            2000 2666      20595360
Brazil      1999 37737    172006362
            2000 80488    174504898
China       1999 212258  1272915272
            2000 213766  1280428583

### 1) Column headers contain values, not labels

In [153]:
df_p

country   cases          population            
year                 1999    2000        1999        2000
0     Afghanistan     745    2666    19987071    20595360
1          Brazil   37737   80488   172006362   174504898
2           China  212258  213766  1272915272  1280428583

In [154]:
df_p.index
df_p.columns = ["country", 1999, 2000, 2001, 2002]
#Noach keine Tidy Data
df_p

,country,1999,2000,2001,2002
0,Afghanistan,745,2666,19987071,20595360
1,Brazil,37737,80488,172006362,174504898
2,China,212258,213766,1272915272,1280428583


In [155]:
# Tidy Data
df_p.melt(id_vars="country", var_name="year", value_name="#Tote")

,country,year,#Tote
0,Afghanistan,1999,745
1,Brazil,1999,37737
2,China,1999,212258
3,Afghanistan,2000,2666
4,Brazil,2000,80488
5,China,2000,213766
6,Afghanistan,2001,19987071
7,Brazil,2001,172006362
8,China,2001,1272915272
9,Afghanistan,2002,20595360


In [156]:
df_melt_add

,country,year,variable,value,cases,population
0,Afghanistan,1999,cases,745,NaN,NaN
1,Afghanistan,2000,cases,2666,NaN,NaN
2,Brazil,1999,cases,37737,NaN,NaN
3,Brazil,2000,cases,80488,NaN,NaN
4,China,1999,cases,212258,NaN,NaN
5,China,2000,cases,213766,NaN,NaN
6,Afghanistan,1999,population,19987071,NaN,NaN
7,Afghanistan,2000,population,20595360,NaN,NaN
8,Brazil,1999,population,172006362,NaN,NaN
9,Brazil,2000,population,174504898,NaN,NaN


In [157]:
### Tidy Data Folie S. 41 3a)
# df["cases"].str.cat(df["population"])  
df_melt_add["fall3"] = df_melt_add["cases"].astype('string').str.cat (df_melt_add["population"].astype('string'), sep='/')
df_melt_add

,country,year,variable,value,cases,population,fall3
0,Afghanistan,1999,cases,745,NaN,NaN,<NA>
1,Afghanistan,2000,cases,2666,NaN,NaN,<NA>
2,Brazil,1999,cases,37737,NaN,NaN,<NA>
3,Brazil,2000,cases,80488,NaN,NaN,<NA>
4,China,1999,cases,212258,NaN,NaN,<NA>
5,China,2000,cases,213766,NaN,NaN,<NA>
6,Afghanistan,1999,population,19987071,NaN,NaN,<NA>
7,Afghanistan,2000,population,20595360,NaN,NaN,<NA>
8,Brazil,1999,population,172006362,NaN,NaN,<NA>
9,Brazil,2000,population,174504898,NaN,NaN,<NA>


In [158]:
df_melt_add["fal13_split"] = df_melt_add["fall3"].str.split('/')
df_melt_add

,country,year,variable,value,cases,population,fall3,fal13_split
0,Afghanistan,1999,cases,745,NaN,NaN,<NA>,<NA>
1,Afghanistan,2000,cases,2666,NaN,NaN,<NA>,<NA>
2,Brazil,1999,cases,37737,NaN,NaN,<NA>,<NA>
3,Brazil,2000,cases,80488,NaN,NaN,<NA>,<NA>
4,China,1999,cases,212258,NaN,NaN,<NA>,<NA>
5,China,2000,cases,213766,NaN,NaN,<NA>,<NA>
6,Afghanistan,1999,population,19987071,NaN,NaN,<NA>,<NA>
7,Afghanistan,2000,population,20595360,NaN,NaN,<NA>,<NA>
8,Brazil,1999,population,172006362,NaN,NaN,<NA>,<NA>
9,Brazil,2000,population,174504898,NaN,NaN,<NA>,<NA>


In [159]:
df_melt_add[["fall3_case", "fall3_pop"]] = df_melt_add["fall3"].str.split('/', expand=True)
df_melt_add

,country,year,variable,value,cases,population,fall3,fal13_split,fall3_case,fall3_pop
0,Afghanistan,1999,cases,745,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1,Afghanistan,2000,cases,2666,NaN,NaN,<NA>,<NA>,<NA>,<NA>
2,Brazil,1999,cases,37737,NaN,NaN,<NA>,<NA>,<NA>,<NA>
3,Brazil,2000,cases,80488,NaN,NaN,<NA>,<NA>,<NA>,<NA>
4,China,1999,cases,212258,NaN,NaN,<NA>,<NA>,<NA>,<NA>
5,China,2000,cases,213766,NaN,NaN,<NA>,<NA>,<NA>,<NA>
6,Afghanistan,1999,population,19987071,NaN,NaN,<NA>,<NA>,<NA>,<NA>
7,Afghanistan,2000,population,20595360,NaN,NaN,<NA>,<NA>,<NA>,<NA>
8,Brazil,1999,population,172006362,NaN,NaN,<NA>,<NA>,<NA>,<NA>
9,Brazil,2000,population,174504898,NaN,NaN,<NA>,<NA>,<NA>,<NA>


3b

In [160]:
data = {
"info" : ["B|C|D", "B|D", "A|c", "B]D", "B|c", "B|C|D"],
"name" : ["Graham Chapman", "John Cleese", "Terry Gilliam", "Eric Idle", "Terry Jones", "Michael Palin"]
}
df = pd.DataFrame(data)
df

,info,name
0,B|C|D,Graham Chapman
1,B|D,John Cleese
2,A|c,Terry Gilliam
3,B]D,Eric Idle
4,B|c,Terry Jones
5,B|C|D,Michael Palin


In [162]:
df[['A', 'B', 'C', 'D']] = df["info"].str.get_dummies('|')

ValueError: Columns must be same length as key

3d

In [163]:
data = {
"matrikel" : [123, 42, 888, 999],
"name" : ["freak", "doe", "lazy", "crazy"],
"course" : ["ERP, DB1, Prog", "DB2", "", pd.NA]
}
students = pd.DataFrame(data)

In [164]:
students["course_list"] = students["course"].str.split(',')

In [165]:
students["course_list"].apply(pd.Series).fillna("Nixda")

,0,1,2
0,ERP,DB1,Prog
1,DB2,Nixda,Nixda
2,,Nixda,Nixda
3,Nixda,Nixda,Nixda


In [167]:
students["course_list"] = students["course"].str.split(',')

In [169]:
students["course_list"].apply(pd. Series).fillna('Nixda')[0].map({'': "Nixda"})

0      NaN
1      NaN
2    Nixda
3      NaN
Name: 0, dtype: object

In [171]:
students["course_list"].apply(pd.Series).fillna('Nixda').replace({'': 'Nixda'})

,0,1,2
0,ERP,DB1,Prog
1,DB2,Nixda,Nixda
2,Nixda,Nixda,Nixda
3,Nixda,Nixda,Nixda


In [173]:
','.join(["sdf", "ösodf"])

'sdf,ösodf'